In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import sqrt
from numpy.linalg import norm
from numpy.linalg import inv
import openpyxl as op

In [41]:
os.chdir("/Users/sifanliu/Dropbox/Dual Sketch/experiments/CV_results")

In [2]:
K_seq = [3, 5, 10]
alpha_seq = [0.3, 1, 3, 10]
tt_ratio_seq = [0.5, 0.7, 0.9]
gamma_seq = [0.1, 0.7, 1.1, 1.7]

In [3]:
def k_fold(X, Y, K=5, shuffel=True):
    n, p = X.shape
    if shuffel:
        data_tilde = np.concatenate([X, Y], axis=1)
        np.random.shuffle(data_tilde)
        X = data_tilde[:, :p]
        Y = data_tilde[:, p]
    batch_size = np.floor(n / K)
    lbd_seq = np.linspace(0.0001, 5, 20)
    err_lbd = np.zeros(20)
    beta_lbd = np.zeros((p, 20))
    for i in range(20):
        lbd = lbd_seq[i]
        err_tot = 0
        beta_cv = np.zeros(p)
        for k in range(K):
            idx_valid = np.arange(k * batch_size, (k + 1) * batch_size, 1, dtype=int)
            X_valid = X[idx_valid, :]
            Y_valid = Y[idx_valid]
            idx_train = list(set(np.arange(0, n, 1, dtype=int)) - set(idx_valid))
            X_train = X[idx_train, :]
            Y_train = Y[idx_train]
            n_train = X_train.shape[0]
            beta_k = np.linalg.inv(X_train.T @ X_train + n_train * lbd * np.identity(p)) @ X_train.T @ Y_train
            beta_cv = beta_cv + beta_k
            pred_err_k = norm(Y_valid - X_valid @ beta_k) ** 2
            err_tot = err_tot + pred_err_k
        err_lbd[i] = err_tot / K
        beta_lbd[:, i] = beta_cv / K
    min_idx = np.argmin(err_lbd)
    lbd_optim = lbd_seq[min_idx]
    beta_optim = beta_lbd[:, min_idx]
    return lbd_optim, beta_optim.reshape(p, 1)

In [103]:
def train_test(X, Y, tt_ratio=0.8, shuffel=True):
    n, p = X.shape
    n_train = np.int(n * tt_ratio)
    n_test = n - n_train
    if shuffel:
        data_tilde = np.concatenate([X, Y], axis=1)
        np.random.shuffle(data_tilde)
        X = data_tilde[:, :p]
        Y = data_tilde[:, p]
    X_train = X[:n_train, :]
    Y_train = Y[:n_train]
    X_test = X[n_train:, :]
    Y_test = Y[n_train:]
    lbd_seq = np.linspace(0.0001, 5, 20)
    err_lbd = np.zeros(20)
    beta_lbd = np.zeros((p, 20))
    for i in range(20):
        lbd = lbd_seq[i]
        beta_train = np.linalg.inv(X_train.T @ X_train + n_train * lbd * np.identity(p)) @ X_train.T @ Y_train
        err_lbd[i] = norm(Y_test - X_test @ beta_train) ** 2
        beta_lbd[:, i] = beta_train
    min_idx = np.argmin(err_lbd)
    lbd_optim = lbd_seq[min_idx]
    beta_optim = beta_lbd[:, min_idx]
    return lbd_optim, beta_optim.reshape(p, 1)


In [4]:
def leave_one_out(X, Y):
    n, p = X.shape
    lbd_seq = np.linspace(0.001, 5, 20)
    err_lbd = np.zeros(20)
    for i in range(20):
        lbd = lbd_seq[i]
        M_lbd = inv(X.T @ X + n * lbd * np.identity(p))
        beta_lbd = M_lbd @ X.T @ Y
        S_lbd = X @ M_lbd @ X.T
        dd = 1 - np.diag(S_lbd)
        nn = Y - X @ beta_lbd
        err_lbd[i] = np.mean((nn / dd) ** 2)
    min_idx = np.argmin(err_lbd)
    lbd_optim = lbd_seq[min_idx]
    return lbd_optim

In [5]:
rep = 10
# result = pd.DataFrame(np.zeros((rep, 7)), columns=['kf_avg', 'kf_refit', 'kf_bic_refit', 'tt', 'tt_refit', 'tt_bic_refit', 'loo'])
result_kf = np.zeros((len(K_seq), rep, 3))
result_tt = np.zeros((len(tt_ratio_seq), rep, 3))
result_loo = np.zeros(rep)

In [108]:
n = 1000
gamma_seq = [1.1]
alpha_seq = [10]
for gamma in gamma_seq:
    for alpha in alpha_seq:
        print("gamma={},alpha={}".format(gamma, alpha))
        p = np.int(n * gamma)
        for r in range(rep):
            X = np.random.randn(n, p)
            beta = np.random.randn(p, 1) * alpha / sqrt(p)
            epsilon = np.random.randn(n, 1)
            Y = X @ beta + epsilon
            X_test = np.random.randn(n, p)
            epsilon_test = np.random.randn(n, 1)
            Y_test = X_test @ beta + epsilon_test
            gram = X.T @ X
            corr = X.T @ Y
            
            # k-fold
            j = 0
            for K in K_seq:
                lbd_kf, beta_kf = k_fold(X, Y, K)
                # result.loc[r, 'kf_avg'] = norm(Y_test - X_test @ beta_kf) ** 2 / n
                result_kf[j, r, 0] = norm(Y_test - X_test @ beta_kf) ** 2 / n
                beta_kf_refit = inv(gram + n * lbd_kf * np.identity(p)) @ corr
                # result[r, 'kf_refit'] = norm(Y_test - X_test @ beta_kf_refit) ** 2 / n
                result_kf[j, r, 1] = norm(Y_test - X_test @ beta_kf_refit) ** 2 / n
                lbd_kf_correct = lbd_kf * (K - 1) / K
                beta_kf_refit_correct = inv(gram + n * lbd_kf_correct * np.identity(p)) @ corr
                # result[r, 'kf_bic_refit'] = norm(Y_test - X_test @ beta_kf_refit_correct) ** 2 / n
                result_kf[j, r, 2] = norm(Y_test - X_test @ beta_kf_refit_correct) ** 2 / n
                j = j + 1
                
            # train-test
            j = 0
            for tt_ratio in tt_ratio_seq:
                lbd_tt, beta_tt = train_test(X, Y, tt_ratio)
                # result.loc[r, 'tt'] = norm(Y_test - X_test @ beta_tt) ** 2 / n
                result_tt[j, r, 0] = norm(Y_test - X_test @ beta_tt) ** 2 / n
                beta_tt_refit = inv(gram + n * lbd_tt * np.identity(p)) @ X.T @ Y
                # result.loc[r, 'tt_refit'] = norm(Y_test - X_test @ beta_tt_refit) ** 2 / n
                result_tt[j, r, 1] = norm(Y_test - X_test @ beta_tt_refit) ** 2 / n
                lbd_tt_correct = lbd_tt * tt_ratio
                beta_tt_refit_correct = inv(gram + n * lbd_tt_correct * np.identity(p)) @ corr
                # result.loc[r, 'tt_bic_refit'] = norm(Y_test - X_test @ beta_tt_refit_correct) ** 2 / n
                result_tt[j, r, 2] = norm(Y_test - X_test @ beta_tt_refit_correct) ** 2 / n
                j = j + 1
                
            # leave one out
            lbd_loo = leave_one_out(X, Y)
            beta_loo_refit = inv(X.T @ X + n * lbd_loo * np.identity(p)) @ corr
        
        # write results
        with open('kf_mean.csv', 'a') as outfile:
            outfile.write('gamma={}_alpha={}\n'.format(gamma, alpha))
            np.savetxt(outfile, np.mean(result_kf, axis=1), fmt='%-7.7f', delimiter=',')
        
        with open('tt_mean.csv', 'a') as outfile:
            np.savetxt(outfile, np.mean(result_tt, axis=1), fmt='%-7.7f', delimiter=',')
        
        # # result.loc[r, 'loo'] = norm(Y_test - X_test @ beta_loo_refit) ** 2 / n
        # result_loo[r] = norm(Y_test - X_test @ beta_loo_refit) ** 2 / n
        with open('loo.csv', 'a') as outfile:
            np.savetxt(outfile, [np.mean(result_tt)], fmt='%-7.7f', delimiter=',')



gamma=1.1,alpha=10


In [54]:
print("k-fold")
print(np.mean(result_kf, axis=1))
print("train-test")
print(np.mean(result_tt, axis=1))
print("loo")
print(np.mean(result_loo))

k-fold
[[1.04614965 1.04548222 1.04322173]
 [1.04324317 1.04322183 1.04324877]
 [1.04327302 1.04322183 1.04313627]]
train-test
[[1.06493999 1.04487717 1.04379967]
 [1.04513832 1.04487717 1.04337807]
 [1.05054368 1.04769689 1.04668436]]
loo
1.0437888597275031


In [83]:
np.mean(result_loo)

0.5089178205174714

In [69]:
op.

array([0.95817492, 1.01724573])

In [36]:
import os